In [1]:
import pandas as pd
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

## Data Ingestion

In [2]:
separation_time = pd.read_csv("data - Separation Time.csv",header=None).values
separation_time

array([[99999,     1,     3,     3,     3,     3,     3,     3,     3,
            3,     1,     1,     3,     3,     1],
       [    1, 99999,     3,     3,     3,     3,     3,     3,     3,
            3,     1,     1,     3,     3,     1],
       [    3,     3, 99999,     2,     2,     2,     2,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2, 99999,     2,     2,     2,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2,     2, 99999,     2,     2,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2,     2,     2, 99999,     2,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2,     2,     2,     2, 99999,     2,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,     2,     2,     2,     2,     2, 99999,     2,
            2,     3,     3,     2,     2,     3],
       [    3,     3,   

In [3]:
landing_time_cost_df = pd.read_csv("data - Landing Time.csv")
landing_time_cost_df

,Airplane ID,Earliest Landing Slot,Target Landing Slot,Latest Landing Slot,Penalty cost per time slot of landing before target,Penalty cost per time slot of landing after target,Size
0,0,3,4,12,10,10,3
1,1,4,6,15,10,10,3
2,2,2,2,11,30,30,2
3,3,2,2,11,30,30,2
4,4,3,3,11,30,30,2
5,5,3,3,12,30,30,2
6,6,3,3,12,30,30,1
7,7,3,3,11,30,30,1
8,8,3,3,12,30,30,1
9,9,3,4,13,30,30,1


In [4]:
airplane_id = landing_time_cost_df["Airplane ID"].values
earliest_landing = landing_time_cost_df["Earliest Landing Slot"].values
target_landing = landing_time_cost_df["Target Landing Slot"].values
latest_landing = landing_time_cost_df["Latest Landing Slot"].values

early_landing_penalty = landing_time_cost_df["Penalty cost per time slot of landing before target"].values
late_landing_penalty = landing_time_cost_df["Penalty cost per time slot of landing after target"].values
airplane_size = landing_time_cost_df["Size"].values

In [5]:
N = len(earliest_landing)
N

15

In [6]:
runway_size = pd.read_csv("data - Runway.csv")['Max Size'].values
runway_size

array([3, 1, 2], dtype=int64)

In [7]:
num_runways = runway_size.shape[0]
num_runways

3

In [8]:
max_airplane_runway_size_diff = max(airplane_size.max() -  runway_size.min(),  runway_size.max() - airplane_size.min())
max_airplane_runway_size_diff

2

## Model

In [9]:
model = ro.Model("Airplane Landing")

### Decision Variables

In [10]:
scheduled_landing = model.dvar(N)
landing_earliness = model.dvar(N)
landing_lateness = model.dvar(N)

#NxN Matrix to determine whether plane i lands before plane j, i != j
i_lands_before_j = model.dvar((N,N), "B")

# NxN Matrix to determine whether plane i lands before plane j, i != j
lands_on_same_runway = model.dvar((N,N), "B")

# Nxnum_runways Matrix to determine whether plane i lands on runway r
runway_allocation = model.dvar((N,num_runways), "B")

### Objective Function

In [11]:
model.min(sum(early_landing_penalty*landing_earliness) + sum(late_landing_penalty*landing_lateness))

### Constraints

In [12]:
# airplane has to land between the latest and earliest possible time
for i in range(len(latest_landing)):
    model.st(earliest_landing[i] <= scheduled_landing[i])
    model.st(scheduled_landing[i] <= latest_landing[i])

# Calculating the earliness & lateness
# Since landing_earliness, landing_lateness > 0
# If scheduled_landing - target_landing >= 0, landing_earliness >=0 & landing_lateness ==0
# If scheduled_landing - target_landing <= 0, landing_earliness ==0 & landing_lateness >=0
model.st(target_landing - scheduled_landing  == landing_earliness - landing_lateness)

M = 99999
for j in range(N):
    for i in range(N):
        if i != j:
            # If 2 planes are scheduled for the same runway, then at least S amt of time should have elapsed between plane i and plane j
            model.st(scheduled_landing[j] - scheduled_landing[i] >= 
                     separation_time[i,j]*lands_on_same_runway[i,j] - M*i_lands_before_j[j,i])
            
            # Every 2 plane must land either before or after and not the same time
            model.st(i_lands_before_j[i,j] + i_lands_before_j[j,i] == 1)
            
            # Defining the link between lands_on_same_runway & runway_allocation
            for r in range(num_runways):
                model.st(lands_on_same_runway[i,j] >= runway_allocation[i,r] + runway_allocation[j,r] - 1)

# airplane size should not be larger than max size of runway
for airplane_index in range(N):
    for runway_index in range(num_runways):
        model.st(airplane_size[airplane_index] - runway_size[runway_index] <= (1 - runway_allocation[airplane_index,runway_index])*max_airplane_runway_size_diff)


# Each airplane can only land on 1 runway
for i in range(N):
    model.st(sum(runway_allocation[i,r] for r in range(num_runways)) == 1)

# Non negative Constraints
model.st(scheduled_landing >= 0,
        landing_earliness >= 0,
        landing_lateness >= 0
        )

In [13]:
model.do_math()

Conic program object:
Number of variables:           541
Continuous/binaries/integers:  46/495/0
---------------------------------------------
Number of linear constraints:  1126
Inequalities/equalities:       886/240
Number of coefficients:        3316
---------------------------------------------
Number of SOC constraints:     0
---------------------------------------------
Number of ExpCone constraints: 0

### Solving

In [14]:
model.solve(grb)

Restricted license - for non-production use only - expires 2024-10-28
Being solved by Gurobi...
Solution status: 2
Running time: 11.1760s


In [15]:
print("Runway allocation:", runway_allocation.get())

Runway allocation: [[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [16]:
runway_allocation.get()

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [17]:
np.argmax(runway_allocation.get(), axis=1) + 1

array([1, 1, 1, 3, 3, 1, 2, 3, 2, 2], dtype=int64)

In [18]:
print("Scheduled landing times:", scheduled_landing.get())

Scheduled landing times: [8. 7. 2. 2. 6. 4. 5. 4. 3. 7.]


In [19]:
print("Earliness:", landing_earliness.get())
print("Lateness:", landing_lateness.get())

Earliness: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Lateness: [4. 1. 0. 0. 3. 1. 2. 1. 0. 3.]


In [20]:
print("Total cost:", model.get())

Total cost: 350.0


In [21]:
model_output = pd.DataFrame(
    {
    'Airplane ID' : airplane_id,
    'Runway':  np.argmax(runway_allocation.get(), axis=1) + 1,
    'Scheduled Landing': scheduled_landing.get(),
    'Earliest Landing' : landing_time_cost_df['Earliest Landing Slot'].values,
    'Latest Landing' : landing_time_cost_df['Latest Landing Slot'].values,
    'Landing Earliness': landing_earliness.get(),
    'Landing Lateness' : landing_lateness.get()
    }).sort_values(['Runway','scheduled_landing']).reset_index(drop=True)
model_output

KeyError: 'scheduled_landing'

# Wrapping the Model as a Function

In [ ]:
def get_optimal_landing(airplane_id, separation_time, earliest_landing, target_landing, latest_landing, early_penalty, late_penalty,
                        runway_size, airplane_size):
    '''
    Prints the optimal landing time and runway allocation for each plane, among other variables.
    
    Parameters:
        airplane_id (numpy.ndarray): an array of the unique ID of each plane
        separation_time (numpy.ndarray): a matrix of separation times when planes land on the same runway
        earliest_landing (numpy.ndarray): an array of the earliest time each plane can land
        target_landing (numpy.ndarray): an array of the target landing time for each plane
        latest_landing (numpy.ndarray): an array of the latest time each plane can land
        early_penalty (numpy.ndarray): an array containing the penalty per unit time of landing each plane early
        late_penalty (numpy.ndarray): an array containing the penalty per unit time of landing each plane late
        runway_size (numpy.ndarray): an array of the size of the available runways
        airplane_size (numpy.ndarray): an array of the size of the available runways
    '''
    
    check_input_shapes(separation_time, len(airplane_id), len(earliest_landing), len(target_landing), len(latest_landing), 
                       len(early_penalty), len(late_penalty), len(airplane_size))
    
    no_of_planes = len(earliest_landing)
    no_of_runways = len(runway_size)
    max_airplace_runway_size_diff = max(airplane_size.max() -  runway_size.min(),  runway_size.max() - airplane_size.min())

    model = ro.Model('Airplane Landing')
    
    # Decision variables
    scheduled_landing = model.dvar(no_of_planes)
    earliness = model.dvar(no_of_planes)
    lateness = model.dvar(no_of_planes)
    lands_before = model.dvar((no_of_planes, no_of_planes), "B")
    lands_on_same_runway = model.dvar((no_of_planes, no_of_planes), "B")
    runway_allocation = model.dvar((no_of_planes, no_of_runways), "B")
    
    # Objective
    model.min(sum(early_penalty*earliness) + sum(late_penalty*lateness))
    
    # Constraints
    model.st(earliest_landing <= scheduled_landing)
    model.st(scheduled_landing <= latest_landing)

    model.st(scheduled_landing - target_landing == earliness - lateness)
    
    M = 99999
    for j in range(no_of_planes):
        for i in range(no_of_planes):
            if i != j:
                model.st(scheduled_landing[j] - scheduled_landing[i] >= 
                         separation_time[i,j]*lands_on_same_runway[i,j] - M*lands_before[j,i])
                
                model.st(lands_before[i,j] + lands_before[j,i] == 1)
                
                for r in range(no_of_runways):
                    model.st(lands_on_same_runway[i,j] >= runway_allocation[i,r] + runway_allocation[j,r] - 1)
    
        model.st(sum(runway_allocation[j,r] for r in range(no_of_runways)) == 1)

        for r in range(no_of_runways):
            model.st(airplane_size[j] - runway_size[r] <= (1 - runway_allocation[j,r])*max_airplace_runway_size_diff)


    model.st(scheduled_landing >= 0,
             earliness >= 0,
             lateness >= 0)
    
    # Get solution
    model.solve(grb)
    
    model_output = pd.DataFrame({
        'Airplace ID' : airplane_id,
        'Runway':  np.argmax(runway_allocation.get(), axis=1) + 1,
        'Scheduled Landing': scheduled_landing.get(),
        'Earliest Landing' : earliest_landing,
        'Latest Landing' : latest_landing,
        'Landing Earliness': earliness.get(),
        'Landing Lateness' : lateness.get()
        }).sort_values(['Runway','Scheduled Landing']).reset_index(drop=True)
    
    return model.get(), model_output

    
def check_input_shapes(separation, *args):
    assert len(set(args)) == 1, "Length of inputs must be equal"
    assert separation.shape == (args[0], args[0]), "Shape of separation must be NxN, where N is the length of each of the other inputs"

In [ ]:
delay_cost, model_output = get_optimal_landing(airplane_id, separation_time, earliest_landing, target_landing, latest_landing, early_landing_penalty, late_landing_penalty, runway_size, airplace_size)

Being solved by Gurobi...
Solution status: 2
Running time: 3.9810s


In [ ]:
print(f'{delay_cost=}')
model_output

delay_cost=350.0


,Aircraft ID,Runway,scheduled_landing,Earliest Landing,Latest Landing,landing_earliness,landing_lateness
0,2,1,2.0,2,11,0.0,0.0
1,5,1,4.0,3,12,0.0,1.0
2,1,1,7.0,4,15,0.0,1.0
3,0,1,8.0,3,12,0.0,4.0
4,8,2,3.0,3,12,0.0,0.0
5,7,2,4.0,3,11,0.0,1.0
6,9,2,7.0,3,13,0.0,3.0
7,3,3,2.0,2,11,0.0,0.0
8,6,3,5.0,3,12,0.0,2.0
9,4,3,6.0,3,11,0.0,3.0
